In [5]:
import sys
sys.path.append(r'/mnt/e/school24/deproject/financial-market-data-pipeline')  # Adjust the path based on the directory structure

import yaml
import time
from etl import GetDataFromAPI, KafkaHandler, split_df_2_smaller_chunk
from datetime import datetime, timedelta
import boto3
import json
import copy
import math

In [7]:
with open('../config/config.yaml', 'r') as file:
        config = yaml.safe_load(file)
        api_key = config['api']['key']
        stocks = config['stocks']

In [8]:
get_data = GetDataFromAPI(api_key)
# all_us_stocks_raw = get_data.get_list_symbols('US')

In [16]:
bs_fn = []
for stock in stocks:
    single_stock = {'symbol' : stock, 'metrics': get_data.get_basic_basic_financials(stock)}
    bs_fn.append(single_stock)

In [17]:
final_fn = split_df_2_smaller_chunk(list_dict = bs_fn, stock_key= 'symbol', cate_key='basic_financial')

In [19]:
# Print the size of each chunk
a=0
for i, chunk in enumerate(final_fn):
    print(f"Size of chunk {i + 1}: {sys.getsizeof(dum_js(chunk))} bytes")
    print(chunk['category'])
    print(len(chunk))
    a += len(chunk)
    print(a)

Size of chunk 1: 970897 bytes
basic_financial_1
6
6
Size of chunk 2: 845872 bytes
basic_financial_2
6
12


In [18]:
def dum_js(v):
    """
    convert to json string to uft8 encoded
    """
    return json.dumps(v).encode('utf-8')

In [5]:
###To run the code faster, collect all stock information into a huge dictionary, check the bytes size, then divide by 1,0000,000 to figure out how many chunks should be separated into.###

#first check the large dictionary size
all_us_stocks = {} 
chunk_size = 1000000
additional_key = {"category": "stock_info_{}"}
for stock in all_us_stocks_raw:
    single_stock = {stock['symbol']: stock}
    all_us_stocks.update(single_stock)
all_stock_size = sys.getsizeof(dum_js(all_us_stocks))
num_chunk_split_into = math.ceil(all_stock_size / (chunk_size - sys.getsizeof(dum_js(additional_key))))
len_each_chunk = math.ceil(len(all_us_stocks_raw) /  num_chunk_split_into)


#split all stock into chunks
all_us_stocks_list = []
stocks_chunk = {}
start_key = 1
key_set = set()
for idx, stock in enumerate(all_us_stocks_raw):
    additional_key = {"category": f"stock_info_{start_key}"}
    # print(additional_key)
    key_set.add(additional_key["category"])

    if len(stocks_chunk) < len_each_chunk - 1 :
        # print(f"true {len(stocks_chunk)}")
        single_stock = {stock['symbol']: stock}
        stocks_chunk.update(single_stock)
    else:
        # print(f"false ")
        stocks_chunk.update(additional_key)
        final_chunk = copy.deepcopy(stocks_chunk)
        all_us_stocks_list.append(final_chunk)
        stocks_chunk.clear()
        start_key += 1
        
#the remain stocks in last chunk
if stocks_chunk:
    stocks_chunk.update(additional_key)
    all_us_stocks_list.append(stocks_chunk)


In [6]:
len(all_us_stocks_raw)
# num_chunk_split_into
# len_each_chunk
# key_set

26668

In [28]:
len(all_us_stocks_list)

7

In [7]:
# Print the size of each chunk
a=0
for i, chunk in enumerate(all_us_stocks_list):
    print(f"Size of chunk {i + 1}: {sys.getsizeof(dum_js(chunk))} bytes")
    print(chunk['category'])
    print(len(chunk))
    a += len(chunk)
    print(a)

Size of chunk 1: 900855 bytes
stock_info_1
3810
3810
Size of chunk 2: 901303 bytes
stock_info_2
3810
7620
Size of chunk 3: 901433 bytes
stock_info_3
3810
11430
Size of chunk 4: 900874 bytes
stock_info_4
3810
15240
Size of chunk 5: 900769 bytes
stock_info_5
3810
19050
Size of chunk 6: 900845 bytes
stock_info_6
3810
22860
Size of chunk 7: 900947 bytes
stock_info_7
3809
26669


In [12]:
company_profile = {"category": "company_info"}
for stock in stocks:
    print(stock)
    company_profile[stock] = get_data.get_company_profile(stock)

AAPL
GOOGL
MSFT
AMZN
NVDA
META
TSLA
V
wMT
MA
JNJ


In [16]:
print(type(dum_js(company_profile)))
print(dum_js(company_profile))

<class 'bytes'>
b'{"category": "company_info", "AAPL": {"country": "US", "currency": "USD", "estimateCurrency": "USD", "exchange": "NASDAQ NMS - GLOBAL MARKET", "finnhubIndustry": "Technology", "ipo": "1980-12-12", "logo": "https://static2.finnhub.io/file/publicdatany/finnhubimage/stock_logo/AAPL.svg", "marketCapitalization": 2660327.1586526455, "name": "Apple Inc", "phone": "14089961010", "shareOutstanding": 15441.88, "ticker": "AAPL", "weburl": "https://www.apple.com/"}, "GOOGL": {"country": "US", "currency": "USD", "estimateCurrency": "USD", "exchange": "NASDAQ NMS - GLOBAL MARKET", "finnhubIndustry": "Media", "ipo": "2004-08-19", "logo": "https://static2.finnhub.io/file/publicdatany/finnhubimage/stock_logo/GOOG.svg", "marketCapitalization": 1881127.2712680658, "name": "Alphabet Inc", "phone": "16502530000", "shareOutstanding": 12433, "ticker": "GOOGL", "weburl": "https://abc.xyz/"}, "MSFT": {"country": "US", "currency": "USD", "estimateCurrency": "USD", "exchange": "NASDAQ NMS - GL